# Generic NeuroEvolution with Neural Networks
## Problem Statement
```
Build a configurable and customizable class that can be used to simulate neuroevolution in any environment.
```
1. Generate Individuals. The individuals here would be Neural Networks.
2. Cross two Individuals to create two more. Crossing is basically swapping weights between the two individuals to create children.
3. Mutate Weights. Change random weights of the two children to some random value.
4. Cost Fuction. This is the part that would be problematic here. The fitness function is dependent on the environment. So I need to pass the fitness function into the class as a callable. OR. I can call the cost functions independently and pass the array of costs to the class. 

In [1]:
import numpy as np
import pandas as pd
import random
import sys
import os
from copy import deepcopy

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation

/opt/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def generate_models(n, layers):
    # Define Layers as [3, 7, 2]
    models = []
    for _ in range(0, n):
        model = Sequential()
        model.add(Dense(input_dim = layers[0], units = layers[1], activation='softmax'))
        for i in range(2, len(layers)):
            model.add(Dense(units = layers[i], activation='softmax'))
        models.append(model)
    
    return models

Given the layer dimensions `layers` and the number of individuals `n` to be generated, `generate_models` generates `n` neural networks set to random dimensions. The layers are all dense for now. Future improvements should include the ability to pass a model and create random model copies out of it.

In [4]:
m = generate_models(2, [3,5,1])

In [5]:
a = (m[1].get_weights())
for weight in a:
    display(weight)
    
a = (m[0].get_weights())
for weight in a:
    display(weight)

array([[-0.03555256,  0.06422722, -0.8460808 , -0.01014441, -0.5269511 ],
       [-0.5366466 ,  0.7905615 , -0.16158521, -0.38890126, -0.70113134],
       [-0.47549668, -0.7120261 , -0.7180643 ,  0.09142244,  0.5848972 ]],
      dtype=float32)

array([0., 0., 0., 0., 0.], dtype=float32)

array([[-0.36137938],
       [-0.42400837],
       [ 0.97113633],
       [ 0.417907  ],
       [-0.9464278 ]], dtype=float32)

array([0.], dtype=float32)

array([[ 0.05865771, -0.04157734, -0.1871615 , -0.7084225 ,  0.33334464],
       [-0.2245804 ,  0.13263476,  0.4403296 , -0.7794607 , -0.5626081 ],
       [ 0.76541656, -0.85100836, -0.1254884 ,  0.19162554,  0.55902845]],
      dtype=float32)

array([0., 0., 0., 0., 0.], dtype=float32)

array([[-0.98808765],
       [-0.8483772 ],
       [ 0.6599841 ],
       [-0.21556973],
       [-0.14797163]], dtype=float32)

array([0.], dtype=float32)

In [6]:
def cross(parent_1, parent_2, crossover_rate, mutation_rate):
    weights1 = parent_1.get_weights()
    weights2 = parent_2.get_weights()
    
    child_weight_1 = deepcopy(weights1)
    
    for i in range(0, len(weights1),2):
        for j in range(0, len(weights1[i])):
            for k in range(0, len(weights1[i][j])):
                if random.uniform(0,1) > crossover_rate:
                    child_weight_1[i][j][k] = weights2[i][j][k]
                    print(i, j, k)
            
                if random.uniform(0,1) < mutation_rate:
                    child_weight_1[i][j][k] += random.uniform(-1, 1)
                
                
    child_1 = Sequential.from_config(parent_1.get_config())
    child_1.set_weights(child_weight_1)
    
    return child_1
        

The crossing works very similar to the algorithm I used in my string matcher experiment. For a given weight in the child, I randomly choose between the respective weights of the two parents

In [7]:
child1 = cross (m[0], m[1], 0.5, 0.1)

0 0 0
0 0 1
0 0 2
0 0 4
0 1 0
0 2 2
0 2 3
0 2 4
2 0 0
2 2 0


In [8]:
child1.get_weights()

[array([[-0.03555256, -0.35645416, -0.8460808 , -0.7084225 , -0.5269511 ],
        [-0.5366466 ,  0.13263476,  0.4403296 , -0.7794607 , -0.5626081 ],
        [ 0.76541656, -0.85100836, -0.7180643 ,  0.09142244,  0.5848972 ]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32),
 array([[-0.36137938],
        [-0.8483772 ],
        [ 0.97113633],
        [-0.67373973],
        [-0.14797163]], dtype=float32),
 array([0.], dtype=float32)]

This seems to be working pretty well too. I randomly choose to swap weights from the two parents based on a `crossover_rate`. The algorithm is more expensive but I hope to see good results. It worked pretty well for the string matcher thing I wrote a while back. Mutation is basically choosing a random value from -1 to 1 based on a `mutation_rate`.  
Okay, the `get_weights` function returns the values for the activation layers as well. And the crossover algorithm randomly mutates these as well. This might be a problem. But theoretically, the activation layer is basically the categorized probabilities at that stage. So, if the algorithm changes these values, it should technically get reupdated to the actual values based on the changed weights. 
Let's see.

## 17/5/2018
I managed to change the crossver function to make it change only the weights and not the activations. It should work fine now.

2 parents => 1 child


In [10]:
def next_generation(current_population, fitness, n_best_individuals, n_total_survivors):
    current_population = [individual for _,individual in sorted(zip(fitness, current_population))]
    survivors = current_population[0:n_best_individuals]
    for _ in range(0, n_total_survivors - n_best_individuals):
        survivors.append(random.randint(n_best_individuals, len(current_population)))
    new_generation = []
    for i in range(0, len(survivors)):
        for j in range(i, len(survivors)):
            if (i == j):
                new_generation.append(survivors[i])
            else:
                new_generation.append(cross(survivors[i], survivors[j], 0.5, 0.2))
    return new_generation

In [12]:
cost_array = [0.8, 0]
next_generation(m, cost_array, 2, 2)

0 0 0
0 0 3
0 0 4
0 1 3
0 1 4
0 2 1
0 2 2
0 2 4
2 1 0
2 2 0


Pretty much everything is working now. I need to convert this to a class that I can use everywhere.

In [15]:
class neuroevolution:
    
    def __init__ (population_size, layers, mutation_rate, n_best_survivors, n_total_survivors, crossover_rate):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.n_best_survivors = n_best_survivors
        self.n_total_survivors = n_total_survivors
        self.crossover_rate = crossover_rate
        self.best_models = []
        
    def init_generation():
        self.current_population = []
        for _ in range(0, self.population_size):
            model = Sequential()
            model.add(Dense(input_dim = self.layers[0], units = self.layers[1], activation='softmax'))
            for i in range(2, len(self.layers)):
                model.add(Dense(units = self.layers[i], activation='softmax'))
            current_population.append(model)
        return current_population
    
    def cross(parent_1, parent_2, crossover_rate, mutation_rate):
        weights1 = parent_1.get_weights()
        weights2 = parent_2.get_weights()
    
        child_weight_1 = deepcopy(weights1)
    
        for i in range(0, len(weights1),2):
            for j in range(0, len(weights1[i])):
                for k in range(0, len(weights1[i][j])):
                    if random.uniform(0,1) > crossover_rate:
                        child_weight_1[i][j][k] = weights2[i][j][k]
                        print(i, j, k)
            
                    if random.uniform(0,1) < mutation_rate:
                        child_weight_1[i][j][k] += random.uniform(-1, 1)
                
                
        child_1 = Sequential.from_config(parent_1.get_config())
        child_1.set_weights(child_weight_1)
    
        return child_1
    
    def next_generation(fitness):
        self.current_population = [individual for _,individual in sorted(zip(fitness, self.current_population))]
        survivors = self.current_population[0:self.n_best_individuals]
        self.best_models.append(self.current_population[0])
        for _ in range(0, self.n_total_survivors - self.n_best_individuals):
            survivors.append(random.randint(self.n_best_individuals, len(self.current_population)))
        new_generation = []
        for i in range(0, len(survivors)):
            for j in range(i, len(survivors)):
                if (i == j):
                    new_generation.append(survivors[i])
                else:
                    new_generation.append(cross(survivors[i], survivors[j], 0.5, 0.2))
        self.current_population = new_generation
        return current_population
    
    def history():
        return self.best_models
    

This should be our final simple class. It's functionalities are kind of limited but it serves my purpose pretty well. It's simple enough to modify the model creation and the mutation to my will. 

**NOTE: THIS CLASS HAS BUGS BECAUSE IT WASN'T TESTED. CHECK THE .py FILE IN THE SAME REPOSITORY WHICH HAS THE BUG FREE ACTUALLY WORKING CLASS.**

The .py version of this class that can be imported in other scripts is available in the same repository